In [1]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from re import sub
from nltk import download
download('punkt')
download('stopwords')

import json
import os
from re import compile, findall, escape
import re
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\herik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\herik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def remove_num(text):
    text = sub(r'\d+', '', text)
    text = sub(r'\s+', ' ',text)
    return text

def remove_punct(text):
    text = sub(r"[!#$%&'()*+,-./:;<=>?@[^_`{|}~]+", ' ',text)
    text = sub(r'\s+', ' ',text)
    return text

def extract_keywords(text):
    tokens = word_tokenize(text)
    keywords = []
    for word in tokens:
        word = word.lower()
        if word not in stopwords.words('portuguese') or word.lower() not in STOP_WORDS:
            keywords.append(word)
    return ' '.join(keywords)

def get_synonyms(text):
    tokens = word_tokenize(text)
    synonyms = []
    for word in tokens:
        for syn in wordnet.synsets(word, lang="por"):
            for lemma in syn.lemmas(lang="por"):
                synonyms.append(lemma.name())
    return synonyms

def remove_accent(text):
    text = sub('[áàãâä]', 'a', sub('[éèêë]', 'e', sub('[íìîï]', 'i', sub('[óòõôö]', 'o', sub('[úùûü]', 'u', text)))))
    text = sub(r'\s+', ' ',text)
    return text

def preprocess_lemma(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmas = []
    for token in tokens:
        lemmas.append(lemmatizer.lemmatize(token))
    lemmas = ' '.join(lemmas)
    return lemmas

def preprocess_stem(text):
    stemmer = SnowballStemmer("portuguese")
    tokens = word_tokenize(text)
    stems = []
    for token in tokens:
        stems.append(stemmer.stem(token))
    stems = ' '.join(stems)
    return stems

def preprocess(text, tipo=None):
    text = remove_punct(text)
    text = remove_num(text)
    text = extract_keywords(text)
    if tipo == 'lemma':
        text = preprocess_lemma(text)
    elif tipo == 'stem':
        text = preprocess_stem(text)
    else:
        pass
    text = remove_accent(text)
    return text

def load_txt(path):
    with open(path,'r', encoding='utf-8') as file:
        linhas = file.readlines()
        lines = []
        for linha in linhas:
            linha = linha.strip()
            if linha != '':
                lines.append(linha)
    return lines

In [64]:
def create_json(json_path, content):
    if os.path.isfile(json_path):
        with open(json_path, 'r+', encoding='utf-8') as f:
            data = json.load(f)
            data.append(content)
            f.seek(0)
            json.dump(data, f, indent=4)
    else:
        with open(json_path, 'w', encoding='utf-8') as f:
            data = [content]
            json.dump(data, f, indent=4)

def check_intervals(lst):
    spans = []
    for current_interval in lst:
        start, end, label = current_interval
        current_span = (start, end, label)
        overlap = False
        for existing_span in spans:
            if existing_span[0] <= start < existing_span[1] or existing_span[0] < end <= existing_span[1]:
                overlap = True
                break
        if not overlap:
            spans.append(current_span)
    corrected_list = [[start, end, label] for start, end, label in spans]
    return corrected_list

def find_words(text, find_tokens):
    result = []
    for token in find_tokens:
        pattern = compile(r'\b{}\b'.format(escape(token)))
        matches = pattern.finditer(text)
        for match in matches:
            dictionary = {
                "text": token,
                "start_index": match.start(),
                "end_index": match.end() - 1,
                "start_position": len(findall(r'\b\w+\b', text[:match.start()])),
                "end_position": (len(findall(r'\b\w+\b', text[:match.start()])) + len(token.split())) - 1
            }
            result.append(dictionary)
    return result

def preparing_data(df, keywords, path, labels_with_texts):
    list_words = []
    for key in keywords:
        words = list(set([preprocess(i, 'lemma') for i in key['keywords'] if i != '']))
        list_words.append(words)
        dict_train = {}
    for i in range(len(labels_with_texts)):
        list_tuple = []
        for text in labels_with_texts[i]["texts"]:
            for id, row in df.iterrows():
                if labels_with_texts[i]["classe"] == row['classe']:
                    label = row['equipament'].upper()
            text = preprocess(text).strip()
            list_find_word = find_words(text,list_words[i])
            list_find = []
            for j in list_find_word:
                if j['text'] != '':
                    list_word_found = [j['start_index'],j['end_index']+1,label]
                    list_find.append(list_word_found)
            list_tuple.append((text, {"entities": list_find}))
        dict_train[label] = list_tuple
        for values in dict_train.values():
            for items in values:
                for inner_values in items[1].values():
                    new_value = check_intervals(inner_values)
                    items[1]['entities'] = new_value
        create_json(path, dict_train)


# def preparing_data(df, keywords, path, labels_with_texts):
#     for key in keywords:
#         words = list(set([preprocess(i, 'lemma') for i in key['keywords'] if i != '']))
#     dict_train = {}
#     for i in range(len(labels_with_texts)):
#         list_tuple = []
#         for text in labels_with_texts[i]["texts"]:
#             text = preprocess(text).strip()
#             list_find_word = find_words(text,words[i])
#             list_find = []
#             for j in list_find_word:
#                 list_word_found = [j['start_index'],j['end_index']+1,label]
#                 list_find.append(list_word_found)
#             list_tuple.append((text, {"entities": list_find}))
#         dict_train[label] = list_tuple

#         for values in dict_train.values():
#             for items in values:
#                 for inner_values in items[1].values():
#                     new_value = check_intervals(inner_values)
#                     items[1]['entities'] = new_value

#         create_json(path, dict_train)


    # dict_train = {}
    # for idx, equipament in enumerate(labels_with_texts['labels'].unique()):
    #     dict_train[equipament] = []
    #     df_filter = labels_with_texts.loc[labels_with_texts['labels'] == equipament]
    #     for id, row in df_filter.iterrows():
    #         label = row['labels'].upper()
    #         text = preprocess(row['samples'], 'lemma').strip()
    #         list_find_word = find_words(text, words[idx])
    #         if list_find_word:
    #             list_find = []
    #             for j in list_find_word:
    #                 list_word_found = [j['start_index'], j['end_index'] + 1, label]
    #                 list_find.append(list_word_found)
    #             dict_train[equipament].append((text, {"entities": list_find}))
    # for values in dict_train.values():
    #     for items in values:
    #         for inner_values in items[1].values():
    #             new_value = check_intervals(inner_values)
    #             items[1]['entities'] = new_value
    # create_json(path, dict_train)




def criar_dataframe(classe, lista):
    data = {'labels': [classe] * len(lista),
            'samples': lista}
    df = pd.DataFrame(data)
    return df

def concat_df(list_df):
    df_concated = pd.concat(list_df, axis=0)
    return df_concated

def preparing_data2(path):
    list_df = []
    count = 0
    for nome_arquivo in os.listdir(path):
        file = f'{path}\{nome_arquivo}'
        lines = load_txt(file)
        if count < 10:
            label = 'bombas'
        elif count > 9 and count < 19:
            label = 'rolamentos'
        elif count > 18 and count < 31:
            label = 'válvulas'
        elif count > 30 and count < 43:
            label = 'acionamentos por corrente'
        elif count > 42 and count < 46:
            label = 'caixas de engrenagens'
        elif count > 45 and count < 54:
            label = 'Sistemas de óleo lubrificante'
        elif count > 53 and count < 70:
            label = 'Acionamentos por correia em V'
        elif count > 69 and count < 74:
            label = 'Sistemas de ventiladores'
        elif count > 73 and count < 79:
            label = 'Purgadores de vapor'
        elif count > 78 and count < 105:
            label = 'Motores elétricos'
        elif count > 104 and count < 107:
            label = 'Contatos elétricos'
        elif count > 106 and count < 110:
            label = 'Disjuntores elétricos de caixa moldada'
        elif count > 109 and count < 113:
            label = 'Circuito magnético'
        elif count > 112 and count < 116:
            label = 'Circuito dielétrico'
        count += 1
        df = criar_dataframe(label,lines)
        list_df.append(df)
    new_df = concat_df(list_df)
    return new_df
        

In [20]:

df_labels = pd.read_excel('classes.xlsx')
display(df_labels)

import json
with open(r'keywords_equipaments.json','r',encoding="utf-8") as f:
    keywords_equipaments = json.load(f)
with open(r'entidades_samples.json','r',encoding="utf-8") as file:
    samples = json.load(file)

,classe,equipament,keywords
0,1,bombas,bomba
1,2,rolamentos,rolamento
2,3,válvulas,válvula
3,4,acionamentos por corrente,"corrente, acionamento corrente"
4,5,caixas de engrenagens,caixa engrenagem
5,6,Sistemas de óleo lubrificante,"reservatório óleo, bomba óleo, filtro óleo, ra..."
6,7,Acionamentos por correia em V,"polias V, correia V, tensionadores V, acoplame..."
7,8,Sistemas de ventiladores,"ventilador, exaustor, soprador, sistema ventil..."
8,9,Purgadores de vapor,Purgadores vapor
9,10,Motores elétricos,Motores elétricos


In [65]:
preparing_data(df_labels, keywords_equipaments, 'json_train_NER.json', samples)

In [66]:
# !python -m spacy download "pt_core_news_sm"
from spacy.util import minibatch, compounding
from spacy import blank, training, load
from pathlib import Path
import random
nlp = load("pt_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


You should consider upgrading via the 'c:\Users\herik\OneDrive\Área de Trabalho\chatbot\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [70]:
def save_model(model, path_model):
    path_model = Path(path_model)
    if not path_model.exists():
        path_model.mkdir()
    model_path = path_model
    model.to_disk(model_path)
    print("Model saved to:", model_path)

def train_model(data_dict, epochs, path_model):
    nlp = blank("pt")
    nlp.add_pipe("ner", name="ner", last=True)
    data = data_dict[0]
    for label in data.keys():
        nlp.get_pipe("ner").add_label(label)
    train_data = []
    for label, examples in data.items():
        for text, annotations in examples:
            train_data.append((text, annotations))
    nlp.begin_training()
    for itn in range(epochs):
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            example_batch = []
            for text, annotation in zip(texts, annotations):
                doc = nlp.make_doc(text)
                example = training.example.Example.from_dict(doc, annotation)
                example_batch.append(example)
            nlp.update(example_batch, losses=losses)
        print("Epoch:", itn+1, "Loss:", losses)
    save_model(nlp, path_model)

In [72]:
with open(r'json_train_NER.json','r',encoding="utf-8") as file:
    data = json.load(file)

train_model(data,20,'model')

Epoch: 1 Loss: {'ner': 383.69433764757844}
Epoch: 2 Loss: {'ner': 123.6943181494562}
Epoch: 3 Loss: {'ner': 38.722802261152765}
Epoch: 4 Loss: {'ner': 16.51127979042253}
Epoch: 5 Loss: {'ner': 9.055934603092245}
Epoch: 6 Loss: {'ner': 5.779707924894765}
Epoch: 7 Loss: {'ner': 6.008568786807641}
Epoch: 8 Loss: {'ner': 3.345335745332085}
Epoch: 9 Loss: {'ner': 47.68549356871944}
Epoch: 10 Loss: {'ner': 46.859574104781764}
Epoch: 11 Loss: {'ner': 2.0948948507165928}
Epoch: 12 Loss: {'ner': 0.06428383184463742}
Epoch: 13 Loss: {'ner': 0.006224407583691102}
Epoch: 14 Loss: {'ner': 0.00044805220314622987}
Epoch: 15 Loss: {'ner': 8.777275355254388e-06}
Epoch: 16 Loss: {'ner': 7.452502889684638e-06}
Epoch: 17 Loss: {'ner': 1.2241198470858317e-06}
Epoch: 18 Loss: {'ner': 7.154116777455006e-07}
Epoch: 19 Loss: {'ner': 6.656823267872591e-07}
Epoch: 20 Loss: {'ner': 2.407977885694445e-07}
Model saved to: model
